## Environment Setup


In [3]:
try:
  import os
  import json
  from pathlib import Path
  import torch
  import numpy as np
  from google.colab import files

  import evaluate
  from transformers import (
      Trainer,
      TrainingArguments,
      DataCollatorForTokenClassification,
      EarlyStoppingCallback,
  )
  import optuna
  import wandb

  from nlpcw.utils import get_dataset, load_model, find_file, load_tokenizer, show_random_elements, tokenize_dataset
except:
  # TODO: temporary hack till colab upgrades pyarrow version
  %pip install -q pyarrow
  %pip install -q git+https://github.com/cogniveon/nlpcw.git

In [4]:
%env WANDB_PROJECT=COMM061-NLP-CW
%env WANDB_LOG_MODEL=end
%env WANDB_SILENT=True
%env TOKENIZERS_PARALLELISM true
wandb.login()

env: WANDB_PROJECT=COMM061-NLP-CW
env: WANDB_LOG_MODEL=end
env: WANDB_SILENT=True
env: TOKENIZERS_PARALLELISM=true


<IPython.core.display.Javascript object>

wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


True

In [5]:
# @title Config
MODEL_NAME = "romainlhardy/roberta-large-finetuned-ner" # @param ["romainlhardy/roberta-large-finetuned-ner","google-bert/bert-base-uncased", "google-bert/bert-large-uncased", "romainlhardy/finetuned-ner", "romainlhardy/bert-finetuned-ner", "pucpr/biobertpt-all"] {"allow-input":true}
CHECKPOINT_PATH = None # @param {type:"raw"}
BATCH_SIZE = 8 # @param {type:"integer"}
NUM_EPOCHS = 20 # @param {type:"integer"}
LEARNING_RATE = 1e-6 # @param {type:"number"}
WEIGHT_DECAY = 0.001 # @param {type:"number"}
HPARAM_SEARCH = True # @param {type:"boolean"}

## Train

In [6]:
dataset, id2label, label2id, num_labels = get_dataset()
label_list = dataset["train"].features["ner_tags"].feature.names  # type: ignore
dataset

Generating train split:   0%|          | 0/1072 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/126 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/153 [00:00<?, ? examples/s]

Map:   0%|          | 0/1072 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1072 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/126 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/153 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 1072
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 126
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 153
    })
})

In [7]:
tokenizer = load_tokenizer(
    exp_or_model_name=MODEL_NAME if CHECKPOINT_PATH == None else CHECKPOINT_PATH,
)

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [8]:
tokenized_dataset = tokenize_dataset(dataset, tokenizer)
tokenized_dataset

Tokenizing dataset:   0%|          | 0/1072 [00:00<?, ? examples/s]

Tokenizing dataset:   0%|          | 0/126 [00:00<?, ? examples/s]

Tokenizing dataset:   0%|          | 0/153 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1072
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 126
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 153
    })
})

In [9]:
metric = evaluate.load("seqeval")
data_collator = DataCollatorForTokenClassification(tokenizer)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    assert results != None
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

args = TrainingArguments(
    output_dir=MODEL_NAME.replace("/", "-") if CHECKPOINT_PATH == None else CHECKPOINT_PATH.replace("/", "-"),
    seed=42,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=25,
    save_total_limit=2,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=WEIGHT_DECAY,
    metric_for_best_model="f1",
    load_best_model_at_end=True,
    report_to="wandb",
)


def model_init(trial):
    model, model_config = load_model(
        exp_or_model_name=MODEL_NAME if CHECKPOINT_PATH == None else CHECKPOINT_PATH,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id,
    )

    return model


class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fn = torch.nn.CrossEntropyLoss()


    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")

        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss = self.loss_fn(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    args=args,
    model_init=model_init,
    train_dataset=tokenized_dataset["train"],  # type: ignore
    eval_dataset=tokenized_dataset["validation"],  # type: ignore
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at romainlhardy/roberta-large-finetuned-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 1024]) in the checkpoint and torch.Size([4, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
if HPARAM_SEARCH:
  def optuna_hp_space(trial):
      return {
          "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-4, log=True),
          "weight_decay": trial.suggest_float("weight_decay", 0.0001, 0.01),
          "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [4, 8, 12]),
          "num_train_epochs": trial.suggest_categorical("num_train_epochs", [10, 15, 20]),
      }

  best_trials = trainer.hyperparameter_search(
      backend="optuna",
      direction="minimize",
      hp_space=optuna_hp_space,
      n_trials=10,
  )
else:
  trainer.train()

trainer.evaluate()
wandb.finish()

[I 2024-08-11 21:39:42,943] A new study created in memory with name: no-name-0022aa82-c071-40ad-a696-6e3c31c29b76
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at romainlhardy/roberta-large-finetuned-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 1024]) in the checkpoint and torch.Size([4, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.296900,0.208048,0.932085,0.929396,0.930739,0.927290
2,0.216900,0.194331,0.949699,0.935675,0.942635,0.938464
3,0.214600,0.198522,0.946945,0.939070,0.942991,0.937834
4,0.159800,0.196777,0.951315,0.945180,0.948238,0.942398
5,0.148700,0.230524,0.953728,0.944501,0.949092,0.942713
6,0.121300,0.222926,0.952291,0.945180,0.948722,0.941926
7,0.122600,0.222748,0.950163,0.941616,0.945870,0.940195
8,0.114800,0.234263,0.950977,0.941616,0.946273,0.940510


[I 2024-08-11 21:44:24,393] Trial 0 finished with value: 3.7793759353364864 and parameters: {'learning_rate': 3.805984355680514e-06, 'weight_decay': 0.0031306772261814587, 'per_device_train_batch_size': 8, 'num_train_epochs': 20}. Best is trial 0 with value: 3.7793759353364864.
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at romainlhardy/roberta-large-finetuned-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 1024]) in the checkpoint and torch.Size([4, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.290400,0.220885,0.944694,0.927699,0.936119,0.929965
2,0.188400,0.189076,0.956779,0.931772,0.944110,0.938307
3,0.129800,0.164312,0.955533,0.948235,0.951870,0.947435
4,0.114900,0.185724,0.957087,0.950102,0.953581,0.946805
5,0.063700,0.202291,0.960069,0.950781,0.955402,0.949953
6,0.053900,0.209984,0.958605,0.947217,0.952877,0.947592
7,0.038000,0.225130,0.963001,0.954175,0.958568,0.952471
8,0.035900,0.258309,0.953298,0.949253,0.951271,0.944917
9,0.024500,0.267030,0.957857,0.952817,0.955331,0.948536
10,0.011300,0.323089,0.961030,0.950102,0.955535,0.948694


[I 2024-08-11 21:51:12,561] Trial 1 finished with value: 3.8153603053223404 and parameters: {'learning_rate': 3.468094632528973e-05, 'weight_decay': 0.005490005039977476, 'per_device_train_batch_size': 12, 'num_train_epochs': 15}. Best is trial 0 with value: 3.7793759353364864.
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at romainlhardy/roberta-large-finetuned-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 1024]) in the checkpoint and torch.Size([4, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

[I 2024-08-11 22:40:29,635] Trial 7 finished with value: 3.782903692865684 and parameters: {'learning_rate': 1.8624733487891571e-06, 'weight_decay': 0.009389605627120396, 'per_device_train_batch_size': 4, 'num_train_epochs': 20}. Best is trial 3 with value: 3.773132301056416.
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at romainlhardy/roberta-large-finetuned-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 1024]) in the checkpoint and torch.Size([4, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.298700,0.217858,0.925299,0.918703,0.921989,0.918634
2,0.218900,0.182022,0.956877,0.933978,0.945289,0.938464
3,0.166900,0.221659,0.939240,0.939240,0.939240,0.933743
4,0.132900,0.173048,0.951244,0.947047,0.949141,0.944130
5,0.082600,0.228011,0.942128,0.939409,0.940767,0.935631
6,0.077700,0.202802,0.954904,0.945180,0.950017,0.945231
7,0.059300,0.288153,0.951712,0.943313,0.947494,0.941297
8,0.049300,0.305264,0.952332,0.942634,0.947458,0.941926
9,0.037000,0.312933,0.950471,0.941276,0.945851,0.940667


[I 2024-08-11 22:46:44,395] Trial 8 finished with value: 3.778266350873935 and parameters: {'learning_rate': 6.588702962738333e-05, 'weight_decay': 0.0056942564533528335, 'per_device_train_batch_size': 12, 'num_train_epochs': 15}. Best is trial 3 with value: 3.773132301056416.
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at romainlhardy/roberta-large-finetuned-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([9, 1024]) in the checkpoint and torch.Size([4, 1024]) in the model instantiated
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.254200,0.214181,0.949549,0.929566,0.939451,0.932169


[I 2024-08-11 22:49:00,484] Trial 9 pruned. 


In [11]:
file_path = find_file(f"romainlhardy-roberta-large-finetuned-ner/run-{best_trials.run_id}", "trainer_state.json")
if file_path:
  metrics = json.load(open(file_path, 'r'))
else:
  print("Metrics not found.")

In [13]:
print(f"Best trial run: {best_trials.run_id}")
print(f"Best trial hyperparameters: {json.dumps(best_trials.hyperparameters, indent=2)}")
print(f"Best trial final F1: {metrics['best_metric']:.3}")

Best trial run: 3
Best trial hyperparameters: {
  "learning_rate": 1.0509220562206556e-06,
  "weight_decay": 0.004573106626696881,
  "per_device_train_batch_size": 8,
  "num_train_epochs": 20
}
Best trial final F1: 0.945


In [ ]:
# @title Download optuna results
zipfile_name = "google-bert_bert-large-uncased_optuna_runs" # @param {"type":"string"}
!zip -rq {zipfile_name}.zip results -x "*.pt" "*.safetensors" "*.bin" "*.pth"
files.download(f"{zipfile_name}.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>